In [1]:
import llm2geneset
import json
import time
import openai

In [2]:
models = ["gpt-3.5-turbo-0125", "gpt-4o-2024-05-13"]
models = ["gpt-3.5-turbo-0125"]
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", 
             "WikiPathway_2023_Human"]
lib_names = ["WikiPathway_2023_Human"]

In [3]:

for model in models:
    for lib_name in lib_names:
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
        start_time = time.time()
        pmids = await llm2geneset.esearch_async(gen_res["descr_cleaned"], "pubmed", 1)
        abstracts = await llm2geneset.efetch_pubmed_async(pmids)
        end_time = time.time()
        gen_res["single_relevant_abstract"] = abstracts
        gen_res["single_relevant_abstract_searchtime"] = end_time - start_time
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)


  2%|██▏                                                                                                                            | 14/801 [00:02<02:09,  6.06it/s]


CancelledError: 

In [32]:
# Generate gene sets for ensembling.
aclient = openai.AsyncClient()
for model in models:
    for lib_name in lib_names:
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
            
        context = []
        for x in gen_res['single_relevant_abstract']:
            if len(x) > 0:
                context.append(x[0]['abstract'])
            else:
                context.append("")

        
        gen_res["llm_single_relevant_abstract"] = await llm2geneset.get_genes_context(aclient,
                                                                                      context,
                                                                                      gen_res["descr_cleaned"], 
                                                                                      model=model)
        
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)

  4%|████▉                                                                                                                          | 31/801 [00:03<00:39, 19.66it/s]

retrying
No code blocks found
List all the known genes directly and indirectly involved in the following biological process or cellular component """Soluble ACE2 Mediated Cell Entry Of SARS CoV 2""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """Soluble ACE2 Mediated Cell Entry Of SARS CoV 2""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {
        "gene": "ACE2"
    },
    {
        "gene": "TMPRSS2"
    },
    {
        "gene": "DPP4"
    },
    {
        "gene": "FURIN"
    },
    {
        "gene": "ADAM17"
    },
    {
        "gene": "B2M"
    },
    {
        "

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [00:46<00:00, 17.18it/s]
